<a href="https://colab.research.google.com/github/TAUforPython/BioMedAI/blob/main/LLM%20multimodal%20image2text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [2]:
import os
from huggingface_hub import login
from google.colab import userdata

os.environ["HF_token"] = userdata.get("HF_token")

login(os.environ["HF_token"])

In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    # Move the model to the GPU
    # model = model.to("cuda")
    print("CUDA is available. Using GPU.")
else:
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


In [4]:
# Load the model and tokenizer
model = AutoModel.from_pretrained("ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1",trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-Llama3-V-2_5:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenization_minicpmv_fast.py:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ContactDoctor/Bio-Medical-MultiModal-Llama-3-8B-V1:
- tokenization_minicpmv_fast.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [11]:
# Load an image and tokenize a text input
#image = Image.open("path/to/image.jpg").convert('RGB')
image = Image.open("medLLM_hippocampus.png").convert('RGB')
text_input = "What is the diagnosis for this patient?"
inputs = tokenizer(text_input, return_tensors="pt")

# Create a message with the image and text input
msgs = [{'role': 'user', 'content': [image, text_input]}]

In [12]:

# Generate a response using the model
res = model.chat(image=image, msgs=msgs, tokenizer=tokenizer, sampling=True, temperature=0.95, stream=True)

# Print the generated response
generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

The diagnosis for this patient is neurofibromatosis type 1 (NF-1). The MRI findings of bilateral optic gliomas, brainstem enlargement, and left subdural hematoma are all consistent with NF-1, a genetic disorder that affects the development of the nervous system.

In [13]:
import textwrap
wrapped_generated_text = textwrap.fill(generated_text, width=100)
print(wrapped_generated_text)

The diagnosis for this patient is neurofibromatosis type 1 (NF-1). The MRI findings of bilateral
optic gliomas, brainstem enlargement, and left subdural hematoma are all consistent with NF-1, a
genetic disorder that affects the development of the nervous system.


In [14]:
!pip install -q git+https://github.com/neuml/txtai#egg=txtai[pipeline]

DEPRECATION: git+https://github.com/neuml/txtai#egg=txtai[pipeline] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━

In [15]:
%%capture

from txtai.pipeline import Translation

# Create translation model
translate = Translation()

In [16]:
translation = translate(wrapped_generated_text, "ru")
translation

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/901k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


'Диагноз этого пациента - нейрофиброматоз типа 1 (NF-1).оптическая глиома, расширение мозговой системы и левая субдуральная гематома соответствуют NF-1, агенетическое расстройство, которое влияет на развитие нервной системы.'

In [17]:
wrapped_generated_text = textwrap.fill(translation, width=100)
print(wrapped_generated_text)

Диагноз этого пациента - нейрофиброматоз типа 1 (NF-1).оптическая глиома, расширение мозговой
системы и левая субдуральная гематома соответствуют NF-1, агенетическое расстройство, которое влияет
на развитие нервной системы.


# Too large file for Colab example

In [ ]:
import os
from huggingface_hub import login
from google.colab import userdata

os.environ["HF_token"] = userdata.get("HF_token")

login(os.environ["HF_token"])

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("BiMediX/BiMediX-Eng")
model = AutoModelForCausalLM.from_pretrained("BiMediX/BiMediX-Eng")

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

...

pytorch_model-00016-of-00019.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00017-of-00019.bin:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.02 MB. The target location /root/.cache/huggingface/hub/models--BiMediX--BiMediX-Eng/blobs only has 2465.76 MB free disk space.
  warnings.warn(


pytorch_model-00018-of-00019.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device